# Download data from CBS and Harmonize
This script allows the download of data from the Statistical Offices of the Netherlands and harmonizes the selected time series

In [2]:
# Import libraries
import os 
import pandas as pd
import geopandas as gpd
import cbsodata
import numpy as np
import sys
import glob

# Import variables and set paths
currrent_path = os.path.dirname(os.path.abspath("__file__"))
sys.path.append(os.path.dirname(os.path.dirname(currrent_path)))
from scripts.variables import base_dir, ancillary_POPdata_folder_path, ancillary_data_folder_path, city, country_Orig, engine
from scripts.mainFunctions.basic import createFolder #, unique
from scripts.mainFunctions.excel_sql import dfTOxls
print(base_dir, ancillary_POPdata_folder_path, ancillary_data_folder_path, city)

c:\FUME\DasymetricMapping c:\FUME\DasymetricMapping/grootams_ProjectData/PopData c:\FUME\DasymetricMapping/grootams_ProjectData/AncillaryData grootams


In [3]:
# Downloading table list from CBS
tables = pd.DataFrame(cbsodata.get_table_list())
tables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4966 entries, 0 to 4965
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Updated              4966 non-null   object
 1   ID                   4966 non-null   int64 
 2   Identifier           4966 non-null   object
 3   Title                4966 non-null   object
 4   ShortTitle           4966 non-null   object
 5   ShortDescription     4966 non-null   object
 6   Summary              4966 non-null   object
 7   Modified             4966 non-null   object
 8   MetaDataModified     4966 non-null   object
 9   ReasonDelivery       4966 non-null   object
 10  ExplanatoryText      4966 non-null   object
 11  OutputStatus         4966 non-null   object
 12  Source               4966 non-null   object
 13  Language             4966 non-null   object
 14  Catalog              4966 non-null   object
 15  Frequency            4966 non-null   object
 16  Period

In [4]:
#Make selection of the above tables
tables = tables[["Identifier", "Title","ShortTitle", "Period"]]

# Downloading a subset for Migration data

## Looking for Migration Data at Neighborhood level (wijken en buurten)

In [5]:
# Search with key words in tables
mask = np.column_stack([tables["ShortTitle"].str.contains("migrat")])
migrat_tables = tables.loc[mask.any(axis=1)]
outTables = base_dir + "/{}_ProjectData/keyTables/"
#createFolder(outTables)
#dfTOxls(outTables, 'MigrationTables', migrat_tables) ##### --->>> It doesn't give any results at neighborhood level <<<--- #####

maskNeigh = np.column_stack([tables["ShortTitle"].str.contains("wijk")])
neigh_tables = tables.loc[maskNeigh.any(axis=1)]
#dfTOxls(outTables, 'NeighborhoodTables', neigh_tables)

From the neighborhood tables, we select those that include the key figures (Kerncijfers wijken en buurten). Some of them are for individual years others for series of years

In [6]:
maskPop = np.column_stack([neigh_tables["ShortTitle"].str.contains("Kerncijfers wijken en buurten")])
pop_tables = neigh_tables.loc[maskPop.any(axis=1)]
pop_tablesSeries = pop_tables[pop_tables.Period.str.contains("-") == True]
pop_tablesYears = pop_tables[pop_tables.Period.str.contains("-") == False]

In [7]:
print(pop_tablesSeries)
print(pop_tablesYears)

     Identifier                                    Title  \
3333   70904ned  Kerncijfers wijken en buurten 2009-2012   
3334   81903NED  Kerncijfers wijken en buurten 2004-2008   

                                   ShortTitle       Period  
3333  Kerncijfers wijken en buurten 2009-2012  2009 - 2012  
3334  Kerncijfers wijken en buurten 2004-2008    2004-2008  
     Identifier                               Title  \
938    85039NED  Kerncijfers wijken en buurten 2021   
939    84799NED  Kerncijfers wijken en buurten 2020   
940    84583NED  Kerncijfers wijken en buurten 2019   
941    84286NED  Kerncijfers wijken en buurten 2018   
942    83765NED  Kerncijfers wijken en buurten 2017   
943    83487NED  Kerncijfers wijken en buurten 2016   
3330   83220NED  Kerncijfers wijken en buurten 2015   
3331   82931NED  Kerncijfers wijken en buurten 2014   
3332   82339NED  Kerncijfers wijken en buurten 2013   
3335   80868ned  Kerncijfers wijken en buurten 2003   
3336   70139NED  Kerncijfers wi

## Downloading a subset for migration
For the period 2004 - 2020, we take only the even years, while for the period before 2004 we select all the available years

In [ ]:
# Download data for separate years (2014 - 2020) + (1995 - 2003)
id = pop_tablesYears["Identifier"].tolist()
rawDataPath = ancillary_POPdata_folder_path + '/rawDataMigration'
createFolder(rawDataPath)
for i in range(len(id)):
    year = pop_tablesYears.iloc[i]["Period"]
    year = int(year)
    if year == 2019 :
        print(id[i],year)
        data = pd.DataFrame(
            cbsodata.get_data("{}".format(id[i])))
        dfTOxls(rawDataPath, "{}_keyFig".format(year), data)   
        """elif year < 2013 :  
        print(id[i],year)
        data = pd.DataFrame(
            cbsodata.get_data("{}".format(id[i])))
        dfTOxls(rawDataPath, "{}_keyFig".format(year), data)   """ 
    else:
        print('Year: {} is not selected'.format(year))  

In [ ]:
# Download data for separate years (2014 - 2020) + (1995 - 2003)
id = pop_tablesYears["Identifier"].tolist()
rawDataPath = ancillary_POPdata_folder_path + '/rawDataMigration'
createFolder(rawDataPath)
for i in range(len(id)):
    year = pop_tablesYears.iloc[i]["Period"]
    year = int(year)
    if year > 2013 and (year% 2) == 0 :
        print(id[i],year)
        data = pd.DataFrame(
            cbsodata.get_data("{}".format(id[i])))
        dfTOxls(rawDataPath, "{}_keyFig".format(year), data)   
    elif year < 2013 :  
        print(id[i],year)
        data = pd.DataFrame(
            cbsodata.get_data("{}".format(id[i])))
        dfTOxls(rawDataPath, "{}_keyFig".format(year), data)    
    else:
        print('Year: {} is not selected'.format(year))      

In [ ]:
# Download data for series of years (2004- 2008) + (2009-2013)
idS = pop_tablesSeries["Identifier"].tolist()
for i in idS:
    metadata = pd.DataFrame(cbsodata.get_meta("{}".format(i), 'Perioden'))
    years= metadata['Title'].tolist()
    data = pd.DataFrame(
        cbsodata.get_data("{}".format(i)))
    print(years)
    for year in years:
        year = int(year)
        if (year% 2) == 0 :
            df = data.loc[data['Perioden'] == '{}'.format(year)]
            dfTOxls(rawDataPath, "{}_keyFig".format(year), df)
        else:
            print('Year: {} is not selected'.format(year))    

In [ ]:
print("##### --->>> DOWNLOAD SUCCESFULLY COMPLETED <<<--- #####")

## Perform join between spatial data and these fields
You need to download the geographic data from here: # https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/

In [ ]:
from zipfile import ZipFile

for file in os.listdir(ancillary_data_folder_path+ "/adm/neighborhoodYears/"):
    name = file.split(".")[0]
    print(file)
    zf = ZipFile(ancillary_data_folder_path + "/adm/neighborhoodYears/" + file, 'r')
    createFolder(ancillary_data_folder_path+ "/adm/neighborhoodYears/"+ name)
    zf.extractall(ancillary_data_folder_path+ "/adm/neighborhoodYears/"+ name)
    zf.close()
    os.remove(file)

In [2]:
# Get all spatial data for neighborhoods in list
listPath=[]
# All files ending with .shp with depth of 2 folder
listA=glob.glob(ancillary_data_folder_path + "/adm/neighborhoodYears/*/buurt_*.shp")
listPath.extend(listA)
# All files ending with .shp with depth of 2 folder
listB = glob.glob(ancillary_data_folder_path + "/adm/neighborhoodYears/*/brt_*.shp") 
listPath.extend(listB)
print(listPath)

['c:\\FUME\\DasymetricMapping/grootams_ProjectData/AncillaryData/adm/neighborhoodYears\\buurt_2004\\buurt_2004_gen.shp', 'c:\\FUME\\DasymetricMapping/grootams_ProjectData/AncillaryData/adm/neighborhoodYears\\buurt_2006\\buurt_2006_gn2.shp', 'c:\\FUME\\DasymetricMapping/grootams_ProjectData/AncillaryData/adm/neighborhoodYears\\buurt_2010\\buurt_2010_v3.shp', 'c:\\FUME\\DasymetricMapping/grootams_ProjectData/AncillaryData/adm/neighborhoodYears\\buurt_2012\\buurt_2012.shp', 'c:\\FUME\\DasymetricMapping/grootams_ProjectData/AncillaryData/adm/neighborhoodYears\\buurt_2014\\buurt_2014.shp', 'c:\\FUME\\DasymetricMapping/grootams_ProjectData/AncillaryData/adm/neighborhoodYears\\buurt_2016\\buurt_2016.shp', 'c:\\FUME\\DasymetricMapping/grootams_ProjectData/AncillaryData/adm/neighborhoodYears\\buurt_2018\\buurt_2018_v3.shp', 'c:\\FUME\\DasymetricMapping/grootams_ProjectData/AncillaryData/adm/neighborhoodYears\\buurt_2020\\buurt_2020_v2.shp', 'c:\\FUME\\DasymetricMapping/grootams_ProjectData/Anci

In [ ]:
# Get only the extend of Groot Amsterdam
extentGrootAms = ancillary_data_folder_path + "/adm/nuts3_grootAms.geojson"
s2 = gpd.read_file(extentGrootAms)

neighDataPath = ancillary_POPdata_folder_path + '/neighData_keyFig'
createFolder(neighDataPath)

In [ ]:
Select "2018_buurt"."BU_CODE", "2018_buurt".geometry from "2018_buurt" ,  "nuts3_grootAms" where
ST_Intersects( ST_Transform("2018_buurt".geometry,3035),  "nuts3_grootAms".geom) 
AND ST_AREA(ST_Intersection(ST_Transform("2018_buurt".geometry,3035),  "nuts3_grootAms".geom) ) > ST_AREA(ST_Transform("2018_buurt".geometry,3035))/2 

In [ ]:
# Join data for years > 2003
for i in listPath:
    year= i.split('\\')[-2].split('_')[-1]
    gdf = gpd.read_file(i)

    gdf = gdf.iloc[: , np.r_[-1, 0:4]]
    gdf = gdf.to_crs('epsg:3035')
    
    inter = gpd.overlay(gdf, s2, how='intersection') 
    path = rawDataPath + "/{}_keyFig.xlsx".format(year)
    df = pd.read_excel(path, header=0)
    ndf = df.iloc[: , 1:] 
    
    if 'Perioden' in df.columns: 
        ndf = ndf.drop(columns=['Perioden'])
    if 'BU_CODE' not in inter.columns:
        print('Key column is renamed')
        inter = inter.rename(columns={'BU_2004':'BU_CODE'})
       
    ngdf = inter.join(ndf.set_index('Codering_3'), on ='BU_CODE')
    ngdf = ngdf.to_crs(3035)
    ngdf.to_file(neighDataPath + "/{}_grootams.gpkg".format(year),driver='GPKG',crs="EPSG:3035")

In [ ]:
# Join data for years<=2003
gdf = gpd.read_file(ancillary_data_folder_path + "/adm/neighborhoodYears/buurt_2004/buurt_2004_gen.shp")
gdf = gdf.iloc[: , np.r_[-1, 0:4]]
gdf = gdf.to_crs('epsg:3035')
inter = gpd.overlay(gdf, s2, how='intersection')
inter.to_file(ancillary_data_folder_path + "/adm/neighborhoodYears/buurt_2004/buurt_2004_grootams.gpkg",driver='GPKG',crs="EPSG:3035")
years_list = [1995, 1997, 1999, 2001, 2003] 

for year in years_list:
    final_year = year - 1
    path = rawDataPath + "/{}_keyFig.xlsx".format(year)
    df = pd.read_excel(path, dtype=object, header=0)
    ndf = df.iloc[: , 1:] 
    
    if 'Codering_3' in ndf.columns:
        print ('All good')
    else:
        for col in ndf.columns:
            if col.startswith("Gemeentecode_"):
                print('Key column is renamed')
                ndf = ndf.rename(columns={"{}".format(col): "Gemeentecode_3"})
            elif col.startswith("Wijkcode_"):
                print('Key column is renamed')
                ndf = ndf.rename(columns={"{}".format(col): "Wijkcode_5"})
            elif col.startswith("Buurtcode_"):
                print('Key column is renamed')
                ndf = ndf.rename(columns={"{}".format(col): "Buurtcode_6"})
            else:
                print('NO RENAMING REQUIRED')
        print("Preparing key column")
        ndf['Codering_3'] = ndf[['Gemeentecode_3','Wijkcode_5', 'Buurtcode_6']].astype(str).apply(lambda x : 'BU{}{}{}'.format(x[0],x[1],x[2]), axis=1)
    
    ndf['Codering_3'] = ndf['Codering_3'].str.replace(' ', '')
    dfTOxls(rawDataPath , "/{}_keyFig".format(year), ndf)
    
    ngdf = inter.join(ndf.set_index('Codering_3'), on ='BU_2004')  
    ngdf = ngdf.rename(columns={"BU_2004": "BU_CODE"})  
    ngdf.to_file(neighDataPath + "/{}_grootams.gpkg".format(final_year),driver='GPKG',crs="EPSG:3035")

In [ ]:
# Convert dtype from object to numeric
listGPKG = glob.glob(neighDataPath + "/*.gpkg")
for file in listGPKG:
    geodf = gpd.read_file(file)
    geodf.iloc[:, 14:-1] = geodf.iloc[:, 14:-1].apply(pd.to_numeric, errors='coerce')
    geodf.to_file(file,driver='GPKG',crs="EPSG:3035")

In [ ]:
gcPath = ancillary_POPdata_folder_path + "/gridCells/2018_dataVectorGrid.geojson"
gc = gpd.read_file(gcPath)
ngPath = ancillary_POPdata_folder_path + "/neighData_keyFig/2020_grootams.gpkg"
ng = gpd.read_file(ngPath)

with open(ancillary_POPdata_folder_path + "/gc_summary.txt",'w') as out:
    gc.info(buf = out)

with open(ancillary_POPdata_folder_path + "/nc_summary.txt",'w') as out:
    ng.info(buf = out)


### Get overview of the data 

In [ ]:
ngPath = ancillary_POPdata_folder_path + "/neighData_keyFig"
"""df = pd.DataFrame(index=range(200))
for file in os.listdir(ngPath):
    print(file)
    if file.endswith(".gpkg"):
        ng = gpd.read_file(ngPath + "/" +file)
        year =file.split("_")[0]
        with open(ngPath + "/nc_summaryL.txt",'a') as out:
            l1= "year: {0} --> {1} columns | {2} rows \n".format(year, ng.shape[1], ng.shape[0])
            out.write(l1)
            ng.info(buf = out)
        with open(ngPath + "/nc_summaryS.txt",'a') as out:
            l2= "Year: {0} --> {1} columns | {2} rows \n".format(year, ng.shape[1], ng.shape[0])
            out.write(l2)
        columns = ng.columns.to_list() 
        
        df['{}'.format(year)] = pd.Series(columns)
        #df = df.apply(lambda col: col.drop_duplicates().reset_index(drop=True))
        
        
df.to_csv(ngPath + "/nc_summaryS.csv")
"""
        

In [ ]:
codes = pd.read_excel(ancillary_POPdata_folder_path + "/neighData_keyFig/abbr_codes.xlsx", header=0)
codes_df = codes.iloc[:,0:2]
for index, row in codes_df.iterrows():
    character = "_" + row['columnName'].split("_")[-1]
    row['columnName'] = row['columnName'].split("{}".format(character))[0]
    print(row['columnName'])

In [ ]:
dictionary = dict(zip(codes_df["columnName"], codes_df["abb"]))
dictionary

In [ ]:
for file in os.listdir(ngPath):
    print(file)
    listCol=[]
    if file.endswith(".gpkg"):
        ng = gpd.read_file(ngPath + "/" +file)
        dataframe = ng.iloc[:, 11:-1]
        keys = []
        values = []
        for col in dataframe.columns:
            char = "_" + col.split("_")[-1]
            columnname = col.split("{}".format(char))[0] 
            keys.append(col)
            values.append(columnname) 
        dictionary1 = dict(zip(keys, values))
        ndf = ng.copy()
        ndf = ndf.rename(columns = dictionary1)
        ng_new = ndf.rename(columns = dictionary)

        listCol = list(set(codes_df["abb"].to_list()))
        listColA = ng.iloc[:, 0:10].columns.to_list()
        listCol.extend(listColA)
        listCol.append('geometry')
        print(listCol)
        ndfL = ng_new[ng_new.columns.intersection(listCol)]
        ndfL = ndfL.loc[:,~ndfL.columns.duplicated()]
        print(ndfL.head(2))
        ndfL.to_file(ngPath + "/cleanSelection/" + file, driver='GPKG',crs="EPSG:3035")
    listCol.clear()
        

# Downloading a subset for Education data <<!!!>>

In [ ]:
# Search with key words in tables
maskEdu = np.column_stack([tables["ShortTitle"].str.contains("Opleidingsniveau")])
edu_tables = tables.loc[maskEdu.any(axis=1)]
dfTOxls(outTables, 'EducationTables', edu_tables) ##### --->>> It does give any results at neighborhood level <<<--- #####

# Downloading a subset for Income data

In [ ]:
# Search with key words in tables
maskInc = np.column_stack([tables["ShortTitle"].str.contains("inkome")])
inc_tables = tables.loc[maskInc.any(axis=1)]
dfTOxls(outTables, 'IncomeTables', inc_tables)

In [ ]:
from google_trans_new import google_translator  
translator  = google_translator()  
Pronounce = translator.translate('สวัสดีจีน',lang_src='th',lang_tgt='zh',pronounce=True)  
print(Pronounce)


mig_ data = pd.DataFrame(
        cbsodata.get_data('37325eng', 
                          filters="WijkenEnBuurten eq 'GM0363    '",
                          select=['Periods','2018']))
print(data.head())

data = pd.DataFrame(
        cbsodata.get_data('37325eng', 
                          filters="WijkenEnBuurten eq 'GM0363    '",
                          select=['Periods','2018']))
print(data.head())

In [ ]:
# Downloading entire dataset (can take up to 30s)
#data = pd.DataFrame(cbsodata.get_data('37325eng', dir="dir_to_save_data"))
# Downloading a subset
data = pd.DataFrame(
        cbsodata.get_data('37325eng', 
                          filters="WijkenEnBuurten eq 'GM0363    '",
                          select=['Periods','2018']))
print(data.head())


In [ ]:
# Downloading entire dataset (can take up to 30s)
data = pd.DataFrame(cbsodata.get_data('83765NED'))
print(data.head())

In [ ]:
# Downloading entire dataset (can take up to 30s)
data = pd.DataFrame(cbsodata.get_data('37556'))
print(data.head())